In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from urllib.parse import quote
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [2]:
url_list=list(['https://front.wemakeprice.com/category/division/2100502','https://front.wemakeprice.com/category/section/3100093','https://front.wemakeprice.com/category/section/3100092'])
cat_name=['김치냉장고','스탠드형','뚜껑형']

In [ ]:
from tqdm import tqdm
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
driver = webdriver.Chrome(service = Service(f"C:/Users/user/anaconda3/Lib/site-packages/chromedriver_autoinstaller/115/chromedriver.exe"), options = chrome_options)

for idx in range(len(cat_name)):
    wm_df=pd.DataFrame()
        # 임시 저장 데이터프레임
    wm_df_final=pd.DataFrame()
        # 최종 데이터프레임
    
    # 리스트 값을 순차적으로 순회하여 변수 지정
    wm_url=url_list[idx]
    wm_cat=cat_name[idx]

    driver.implicitly_wait(2)

    driver.get(wm_url)
        # url 방문
    driver.implicitly_wait(2)

    for pg in range(2,12):
        time.sleep(3)
        real_pg=pg-1
            # (2,12) -> (1,11) : 10페이지까지만 수집
        print(pg)

        for i in tqdm(range(1,100)):
                # 한페이지에 상품 100개 존재
            try:
                # url수집
                ele = driver.find_element(By.XPATH, f'//*[@id="list_lists"]/div/div[{i}]/a')
                urls = ele.get_attribute('href')

                # 상품명 수집
                prd=driver.find_element(By.XPATH,f'//*[@id="list_lists"]/div/div[{i}]/a/div/div[2]/div[2]/p')
                prdt_nm=prd.text


                new_df=pd.DataFrame([[prdt_nm,urls,real_pg]],columns=['상품명','url','페이지'])
                wm_df=pd.concat([wm_df,new_df])
            except:
                break
        
        wm_df_final=pd.concat([wm_df_final,wm_df])
        
        # 다음페이지 넘어가기
        try :
            driver.find_element(By.XPATH,f'//*[@id="_contents"]/div/div/div[2]/div[2]/div[5]/a[{pg}]').click()
        except:
            break

    wm_df_final = wm_df_final.reset_index(drop=True)

    # 파일로 저장
    if idx==0:
        file_name='G://다른 컴퓨터//내 MacBook Air//MC//위메프/depth3//' + wm_cat + '.csv'
    else:
        file_name='G://다른 컴퓨터//내 MacBook Air//MC//위메프//depth4//' + wm_cat + '.csv'

    wm_df.to_csv(file_name,index=False,encoding='utf-8-sig')
    print(f'{"-"*30} {wm_cat} 수집 완료 {"-"*30}')